# Deepchem을 활용한 Tox21 분자독성예측 예제

- Deepchem라이브러리를 활용하여 Tox21 데이터세트에 포함된 분자의 독성을 예측해보는 예제
- Deepchem은 텐서플로우 기반으로 신약개발 분야에 활용되는 머신러닝/딥러닝 라이브러리(=패키지)
- 예제를 통하여 머신러닝/딥러닝을 어떻게 실제 세계의 문제에 적용하는지 접근법과 딥러닝 모델의 구체적인 활용법을 이해

## 데이터세트와 도메인 확인

- Deepchem 라이브러리에는 Tox21데이터셋과 이에 사용할 수 있는 딥러닝 모델을 제공함
- Tox21데이터셋이란 약물(분자)의 독성예측과 관련된 표적 단백질의 실험 데이터
- dc.molnet.load_tox21()을 사용하여 아래와 같이 Task, Dataset, Transformer의 3가지 값을 불러올 수 있음

In [ ]:
# 가상환경에 deepchem 라이브러리 설치 필요
# pip install deepchem

In [2]:
import numpy as np
import deepchem as dc
import torch
from collections import OrderedDict
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # 해당 조의 GPU 번호로 변경

tox21_tasks, tox21_datasets, transformers = dc.molnet.load_tox21()

2023-08-31 17:05:29.531861: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 17:05:29.648819: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-31 17:05:29.674775: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-31 17:05:30.217584: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

### Task (표적 단백질)

- Task는 아래 12가지 표적 단백질로 구성
- 표적 단백질은 잠재적으로 신약에 활용될 수 있는 분자와 결합시 독성 반응을 보이는 것으로 여겨짐

In [3]:
tox21_tasks

['NR-AR',
 'NR-AR-LBD',
 'NR-AhR',
 'NR-Aromatase',
 'NR-ER',
 'NR-ER-LBD',
 'NR-PPAR-gamma',
 'SR-ARE',
 'SR-ATAD5',
 'SR-HSE',
 'SR-MMP',
 'SR-p53']

### 데이터 확인

- tox21_datasets는 train, valid, test의 3가지 데이터셋으로 구성됨
- 각 데이터셋에는 X, y, w 벡터가 존재하며 .shape 명령으로 구조를 확인 가능
- X벡터 = 학습 또는 추론에 사용할 feature(특징), 각 샘플은 분자의 FingerPrint
- y벡터 = 학습 또는 추론결과인 정답, 참값등, 각 샘플의 레이블 12개는 표적 단백질 12종과의 결합 정도를 의미
- w벡터 = 가중치(weight)값

In [4]:
train_dataset, valid_dataset, test_dataset = tox21_datasets

In [5]:
train_dataset.X, train_dataset.ids

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]]),
 array(['CC(O)(P(=O)(O)O)P(=O)(O)O',
        'CC(C)(C)OOC(C)(C)CCC(C)(C)OOC(C)(C)C',
        'OC[C@H](O)[C@@H](O)[C@H](O)CO', ...,
        'O=C1OC(OC(=O)c2cccnc2Nc2cccc(C(F)(F)F)c2)c2ccccc21',
        'CC(=O)C1(C)CC2=C(CCCC2(C)C)CC1C',
        'CC(C)CCC[C@@H](C)[C@H]1CC(=O)C2=C3CC[C@H]4C[C@@H](O)CC[C@]4(C)[C@H]3CC[C@@]21C'],
       dtype=object))

In [6]:
print(train_dataset.X.shape)
print(valid_dataset.X.shape)
print(test_dataset.X.shape)

(6264, 1024)
(783, 1024)
(784, 1024)


In [7]:
print(train_dataset.y.shape)
print(valid_dataset.y.shape)
print(test_dataset.y.shape)

(6264, 12)
(783, 12)
(784, 12)


### 불완전한 데이터세트의 적용(가중치를 통한 걸러내기)

- Tox21 데이터셋은 모든 분자와 단백질간의 생화학적 분석이 완료되어있지 않아 결측치 존재
- 모델 학습시 이러한 결측치는 제외되어야 함
- 각 분자 샘플의 가중치를 기록하는 w벡터를 활용하여 해결
- w벡터에는 손실함수를 계산시 샘플과 task에 곱하여 합산하는 가중치값이 포함
- 결측치가 있는 샘플의 경우, 가중치값이 0이므로 손실함수에 대한 영향없이 무시  
(tox21데이터는 지속적으로 갱신되므로 변할 수 있음)

In [8]:
# np.count_nonzero()를 통해 0인 가중치가 얼마나 있는지 확인하기
print(train_dataset.w.shape)
print(np.count_nonzero(train_dataset.w))
print(np.count_nonzero(train_dataset.w==0))

(6264, 12)
63647
11521


### 데이터 사용을 위한 Transformer(변환기)

- transformers 객체는 원본 데이터셋을 변환시켜주는 tool을 포함
- tox21의 분자 대부분은 표적 단백질에 결합하지 않는 데이터임
- 따라서 y벡터의 레이블 대부분이(약 90%) 결합되지 않음을 나타내는 0으로 채워짐
- 항상 결과를 0으로만 예측하는 모델은 정확도가 90%로 측정될 수 있음
- 이렇게 불균형한 데이터를 보완할 수 있도록 가중치 행렬을 조정해주는 Balancing transformer 사용
- 각 클래스(분류 목표)에 할당된 총 가중치가 동일하도록 개별 데이터들의 가중치를 조정

In [9]:
transformers[0].weights

[[1.0450224215246637, 23.211155378486055],
 [1.036325992847732, 28.528497409326423],
 [1.1250265336446614, 8.99830220713073],
 [1.045414847161572, 23.01923076923077],
 [1.1460775473399458, 7.845679012345679],
 [1.056211354693648, 18.79],
 [1.0255516840882695, 40.13636363636363],
 [1.1726791726791728, 6.79108635097493],
 [1.035385448636938, 29.260204081632654],
 [1.0557650327445922, 18.93238434163701],
 [1.1746499631540162, 6.725738396624473],
 [1.05288369419429, 19.909420289855074]]

# 중앙 집중식 학습

## 학습 모델 불러오기

- 학습 모델로 사용하는 MultitaskClassifier는 다중작업분류기임
- 모든 샘플(x)에 대해 여러개의 레이블(y값은 12개)이 있는 다중 분류 문제를 해결하는데 사용
- n_tasks = 분류해야할 작업의 갯수 (12개)
- n_features = 입력될 feature의 갯수 (x벡터 크기인 1024)
- layer_sizes = hidden레이어의 갯수(1)와 너비(1000)  

In [10]:
model = dc.models.MultitaskClassifier(n_tasks=12, n_features=1024, layer_sizes=[1000])

## 모델 학습

- 대부분의 머신러닝 라이브러리들은 모델 학습을 함수화한 fit 함수를 지원
- 위에서 지정한 모델에 train_dataset을 사용하여 10 epoch 학습함
- Epoch란, 신경망 모델에 대하여 전체 데이터셋을 모두 사용하여 순전파, 역전파를 모두 진행하고 학습과정을 완료하였다는 의미

In [16]:
# 방법 1 : 중간 결과물 출력 없이 학습
model.fit(train_dataset, nb_epoch=10)

0.49527231852213544

In [90]:
# 방법 2 : 학습 도중 결과를 저장 및 출력
train_roc_list = []
val_roc_list = []
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)
for i in range(10):
    model.fit(train_dataset, nb_epoch=1)
    train_roc = model.evaluate(train_dataset, [metric])
    val_roc = model.evaluate(valid_dataset, [metric])
    print('Epoch %d:' % (i), end=' ')
    print(train_roc, val_roc)
    train_roc_list.append(train_roc['mean-roc_auc_score'])
    val_roc_list.append(val_roc['mean-roc_auc_score'])

Epoch 0: {'mean-roc_auc_score': 0.8528758574822146} {'mean-roc_auc_score': 0.6973176394472445}
Epoch 1: {'mean-roc_auc_score': 0.8969103174395134} {'mean-roc_auc_score': 0.7069945555737336}
Epoch 2: {'mean-roc_auc_score': 0.9181007869207068} {'mean-roc_auc_score': 0.7135828426629814}
Epoch 3: {'mean-roc_auc_score': 0.928979753667809} {'mean-roc_auc_score': 0.7123335869996831}
Epoch 4: {'mean-roc_auc_score': 0.9379828321487323} {'mean-roc_auc_score': 0.7137258884556483}
Epoch 5: {'mean-roc_auc_score': 0.9447339499353494} {'mean-roc_auc_score': 0.712208815909228}
Epoch 6: {'mean-roc_auc_score': 0.9484793543577533} {'mean-roc_auc_score': 0.71281656075499}
Epoch 7: {'mean-roc_auc_score': 0.9512847462497472} {'mean-roc_auc_score': 0.7147909310334475}
Epoch 8: {'mean-roc_auc_score': 0.9561678266308412} {'mean-roc_auc_score': 0.7112200703290289}
Epoch 9: {'mean-roc_auc_score': 0.9580589632446084} {'mean-roc_auc_score': 0.7137912728152257}


## 모델의 성능 평가

- 학습을 모두 마친 모델은 성능 평가 과정을 거쳐야 함
- 성능평가를 위하여 평가 지표(Metric)을 설정
- 본 예제는 분류 문제이므로, ROC_AUC 점수를 평가지표로 사용하되, 다중 분류이므로 각 분류 점수의 평균값을 사용

In [86]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)

- 위 fit과 같이 evaluate를 통해 학습된 모델에 대해 평가를 진행하되, 이미 학습했던 데이터는 사용하지 않아야 함
- 미리 분리해둔 test 데이터를 활용하면 객관적인 모델 평가가 가능
- train데이터와 test데이터 모두 점수가 높다면 모델이 일반화(Generalization)가 잘 된것으로 평가
- 일반화(Generalization) =  모델이 학습한 데이터 외에도 새로운 데이터에 대한 추론 성능이 높음

In [91]:
test_scores = model.evaluate(test_dataset, [metric], transformers)
print(test_scores)

{'mean-roc_auc_score': 0.6812142854830853}


# 연합학습을 위한 데이터 분할 및 저장

In [9]:
# 하나의 리스트를 n개로 분할하는 함수 정의
import math
def list_split(arr, n):
    num = math.ceil(len(arr) / n)
    return [arr[i: i + num] for i in range(0, len(arr), num)]

In [22]:
# 각 클라이언트가 학습하기위한 데이터 분할
# 해당 예제에서는 3개의 클라이언트를 연합학습
num_clients = 6
x_train_list, y_train_list = map(list_split, (train_dataset.X, train_dataset.y), (num_clients, num_clients))
x_val_list, y_val_list = map(list_split, (valid_dataset.X, valid_dataset.y), (num_clients, num_clients))


In [23]:
def save_datas(file_name:str, data:list):
    try: os.mkdir('./data')
    except: pass
    for i in range(len(data)):
        try: os.mkdir(f'./data/client{i}')
        except: pass
        np.save(f'./data/client{i}/{file_name}.npy', data[i])

In [24]:
file_name_list = ['x_train', 'y_train', 'x_val', 'y_val']
data_list = [x_train_list, y_train_list, x_val_list, y_val_list]

for f_name, d_list in zip(file_name_list, data_list):
    save_datas(f_name, d_list)

# 연합학습 코드
- flower에 모델을 탑재하기 위해 DeepChem에서 제공하는 모델이 어떤 프레임워크(Pytorch, TensorFlow)를 사용했는지 확인
- 모델은 DeepChem에서 불러온 모델 클래스 내부의 model이라는 변수에 저장되어 있음
- 사용된 모델의 프레임워크에 맞게 flower에 탑재

In [14]:
# model 프레임워크 확인
dc_model = dc.models.MultitaskClassifier(n_tasks=12, n_features=1024, layer_sizes=[1000])
print(dc_model.model)
print(type(dc_model.model))

PytorchImpl(
  (layers): ModuleList(
    (0): Linear(in_features=1024, out_features=1000, bias=True)
  )
  (output_layer): Linear(in_features=1000, out_features=24, bias=True)
)
<class 'deepchem.models.fcnet.MultitaskClassifier'>


### 해당 예제에서는 Pytorch 모델을 사용했기에 Pytorch 모델에 맞게 Flower client 코드 작성

# 서버 실행

In [2]:
import flwr as fl
import os
import numpy as np
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # 해당 조의 GPU 번호로 변경


fraction_fit=1
fraction_eval=1
min_fit_clients=5
min_eval_clients=5
min_available_clients=5
num_rounds=20

def evaluate_metrics_aggregation_fn(eval_metrics):
    data_len = sum([num for num, met in eval_metrics])
    acc = sum([num*met['mean-roc_auc_score'] for num, met in eval_metrics])/data_len
    return {'mean-roc_auc_score' : acc}

strategy = fl.server.strategy.FedAvg(
    fraction_fit=fraction_fit,                    # 훈련을 위해서 사용 가능한 클라이언트의 100% 이용
    fraction_evaluate=fraction_eval,              # 평가를 위해서 사용 가능한 클라이언트의 100% 이용
    min_fit_clients=min_fit_clients,              # 훈련을 위해서는 적어도 5개 이상의 클라이언트가 필요
    min_evaluate_clients=min_eval_clients,        # 평가를 위해서는 적어도 5개 이상의 클라이언트가 필요
    min_available_clients=min_available_clients,  # 사용 가능한 클라이언트의 수가 5 될 때까지 대기
    evaluate_metrics_aggregation_fn=evaluate_metrics_aggregation_fn,
)

print('server start!')
output = fl.server.start_server(config=fl.server.ServerConfig(num_rounds=num_rounds), strategy=strategy)
output

INFO flwr 2023-08-28 21:21:09,912 | app.py:148 | Starting Flower server, config: ServerConfig(num_rounds=20, round_timeout=None)
INFO flwr 2023-08-28 21:21:09,920 | app.py:168 | Flower ECE: gRPC server running (20 rounds), SSL is disabled
INFO flwr 2023-08-28 21:21:09,921 | server.py:86 | Initializing global parameters
INFO flwr 2023-08-28 21:21:09,922 | server.py:273 | Requesting initial parameters from one random client


server start!


INFO flwr 2023-08-28 21:21:16,953 | server.py:277 | Received initial parameters from one random client
INFO flwr 2023-08-28 21:21:16,954 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-08-28 21:21:16,955 | server.py:101 | FL starting
DEBUG flwr 2023-08-28 21:21:17,034 | server.py:218 | fit_round 1: strategy sampled 5 clients (out of 5)
DEBUG flwr 2023-08-28 21:21:17,861 | server.py:232 | fit_round 1 received 5 results and 0 failures
WARNING flwr 2023-08-28 21:21:17,888 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-08-28 21:21:17,889 | server.py:168 | evaluate_round 1: strategy sampled 6 clients (out of 6)
DEBUG flwr 2023-08-28 21:21:18,426 | server.py:182 | evaluate_round 1 received 6 results and 0 failures
DEBUG flwr 2023-08-28 21:21:18,427 | server.py:218 | fit_round 2: strategy sampled 6 clients (out of 6)
DEBUG flwr 2023-08-28 21:21:18,628 | server.py:232 | fit_round 2 received 6 results and 0 failures
DEBUG flwr 2023-08-28 21:21:18,663 | se

DEBUG flwr 2023-08-28 21:21:23,443 | server.py:218 | fit_round 20: strategy sampled 6 clients (out of 6)
DEBUG flwr 2023-08-28 21:21:23,615 | server.py:232 | fit_round 20 received 6 results and 0 failures
DEBUG flwr 2023-08-28 21:21:23,655 | server.py:168 | evaluate_round 20: strategy sampled 6 clients (out of 6)
DEBUG flwr 2023-08-28 21:21:23,730 | server.py:182 | evaluate_round 20 received 6 results and 0 failures
INFO flwr 2023-08-28 21:21:23,730 | server.py:147 | FL finished in 6.774742648995016
INFO flwr 2023-08-28 21:21:23,731 | app.py:218 | app_fit: losses_distributed [(1, 0.10000000149011612), (2, 0.10000000149011612), (3, 0.10000000149011612), (4, 0.10000000149011612), (5, 0.10000000149011612), (6, 0.10000000149011612), (7, 0.10000000149011612), (8, 0.10000000149011612), (9, 0.10000000149011612), (10, 0.10000000149011612), (11, 0.10000000149011612), (12, 0.10000000149011612), (13, 0.10000000149011612), (14, 0.10000000149011612), (15, 0.10000000149011612), (16, 0.10000000149011

History (loss, distributed):
	round 1: 0.10000000149011612
	round 2: 0.10000000149011612
	round 3: 0.10000000149011612
	round 4: 0.10000000149011612
	round 5: 0.10000000149011612
	round 6: 0.10000000149011612
	round 7: 0.10000000149011612
	round 8: 0.10000000149011612
	round 9: 0.10000000149011612
	round 10: 0.10000000149011612
	round 11: 0.10000000149011612
	round 12: 0.10000000149011612
	round 13: 0.10000000149011612
	round 14: 0.10000000149011612
	round 15: 0.10000000149011612
	round 16: 0.10000000149011612
	round 17: 0.10000000149011612
	round 18: 0.10000000149011612
	round 19: 0.10000000149011612
	round 20: 0.10000000149011612
History (metrics, distributed, evaluate):
{'mean-roc_auc_score': [(1, 0.6019952855367567), (2, 0.618600876587328), (3, 0.6345508008090812), (4, 0.6494426340712532), (5, 0.6584992695159299), (6, 0.6677232146792814), (7, 0.6755920217092644), (8, 0.6813067373515415), (9, 0.6858426106260785), (10, 0.688675725948092), (11, 0.691090928339881), (12, 0.6927305709057